In [1]:
using HDF5
using LinearAlgebra

In [2]:
global const fid::HDF5.File          = h5open("/home/nabil/work/projects/hrfile_hdf5_converter/cmake-build-debug/wann.hdf5", "r") 
global const nw::Int64               = read(fid["nw"])
global const nr::Int64               = read(fid["nr"]);

global const h::Array{ComplexF64, 3} = reshape(read(fid["reH"]) + im * read(fid["imH"]), (nr, nw, nw));
global const r::Matrix{ComplexF64}   = reshape(read(fid["rvecs"]), (nr, 3));

close(fid)


In [3]:
# Lattice Constants 
global const A₁::Vector{Float64} = [3.9413692012334867, -0.0000000000000000, 0.0000000000000000] 
global const A₂::Vector{Float64} = [0.0000000000000000,  3.9413692012334867, 0.0000000000000000]
global const A₃::Vector{Float64} = [0.0000000000000000, 0.0000000000000000,  3.9413692012334867]
global const A::Matrix{Float64}  = hcat(A₁, A₂, A₃)'  
global const G::Matrix{Float64}  = 2π * inv(A)'
global const G₁::Vector{Float64} = G[1, :]
global const G₂::Vector{Float64} = G[2, :]
global const G₃::Vector{Float64} = G[3, :];

In [4]:
# K-Path 
global const nodes::Vector{Vector{Float64}} = [[0.0, 0.0, 0.0],
         [0.5, 0.5, 0.0],
         [0.5, 0.0, 0.0],
         [0.0, 0.0, 0.0]]

4-element Vector{Vector{Float64}}:
 [0.0, 0.0, 0.0]
 [0.5, 0.5, 0.0]
 [0.5, 0.0, 0.0]
 [0.0, 0.0, 0.0]

In [5]:
function kpath_connect_nodes_red(nodes, A₁, A₂, A₃, nk)
    ks_red = Vector{Vector{Float64}}()
    for ik ∈ 1 : length(nodes) - 1
        start_node = nodes[ik]
        end_node   = nodes[ik + 1]
        pts        = LinRange(start_node, end_node, nk)
        if ik != length(nodes) - 1
            pts = pts[1 : end - 1]
        end
        append!(ks_red, pts)
    end
    reduce(hcat, ks_red)'
end

kpath_connect_nodes_red (generic function with 1 method)

In [14]:
global const ks_red::Matrix{Float64} = kpath_connect_nodes_red(nodes, A₁, A₂, A₃, 100);


In [18]:
# Compute e^ikR Matrix 
function calculate_eikR_mat(ks_red::Matrix{Float64}, Rs::Matrix{ComplexF64}, nk, nr)
    eikR_mat::Matrix{ComplexF64} = zeros(ComplexF64, nk, nr)
    @inbounds Threads.@threads for ik = 1 : nk
        @inbounds for iR = 1 : nr
            eikR_mat[ik, iR] = cispi(2 * dot(ks_red[ik, :], @view Rs[iR, :]))
        end
    end
    eikR_mat
end

@inline function ek(h, eikR, nw, nr)
    Rsum::Matrix{ComplexF64} = zeros(ComplexF64, nw, nw)
    @inbounds Threads.@threads for iR ∈ 1 : size(eikR, 2)
        @inbounds for ik ∈ 1 : size(eikR, 1)
            @inbounds for α = 1 : nw 
                @inbounds for β = 1 : nw 
                    Rsum[α, β] += h[iR][α][β] * eikR[ik, iR]
                end
            end
        end
        
    return hsum
end

hk (generic function with 1 method)

In [21]:
global const eikR_mat::Matrix{ComplexF64} = calculate_eikR_mat(ks_red, r, size(ks_red, 1), nr);

In [47]:
r

1043×3 Matrix{ComplexF64}:
 -7.0+0.0im   1.0+0.0im   2.0+0.0im
 -4.0+0.0im  -2.0+0.0im  -1.0+0.0im
 -3.0+0.0im  -2.0+0.0im   2.0+0.0im
 -7.0+0.0im   2.0+0.0im   2.0+0.0im
 -4.0+0.0im  -2.0+0.0im   0.0+0.0im
 -2.0+0.0im  -2.0+0.0im   2.0+0.0im
 -7.0+0.0im   3.0+0.0im   2.0+0.0im
 -4.0+0.0im  -2.0+0.0im   1.0+0.0im
 -1.0+0.0im  -1.0+0.0im   2.0+0.0im
 -7.0+0.0im  -3.0+0.0im   2.0+0.0im
     ⋮                   
 -2.0+0.0im   1.0+0.0im   7.0+0.0im
 -2.0+0.0im   3.0+0.0im   4.0+0.0im
 -2.0+0.0im   2.0+0.0im   1.0+0.0im
 -1.0+0.0im   2.0+0.0im   7.0+0.0im
 -2.0+0.0im  -3.0+0.0im   4.0+0.0im
 -2.0+0.0im   2.0+0.0im   2.0+0.0im
  0.0+0.0im   2.0+0.0im   7.0+0.0im
 -2.0+0.0im  -2.0+0.0im   4.0+0.0im
 -2.0+0.0im   2.0+0.0im   3.0+0.0im